<a href="https://colab.research.google.com/github/sameh00alsharawy/machine-learning-/blob/master/arabic_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
!gdown --id '1EbAUsvp7_d46k_AJo97c_Kkq2W92Szq6'
!gdown --id '1s675Wt46d0-TqoHPqgM46-kVa4E8_m-Y'
!gdown --id '1NKngN4pBT7nV9y6ALW6vI-w972r918-H'
!gdown --id '1MnyTd18BaP-KVNc2T_q6lmmek5KGVdVo'

Downloading...
From: https://drive.google.com/uc?id=1EbAUsvp7_d46k_AJo97c_Kkq2W92Szq6
To: /content/2class-balanced-test.txt
100% 22.5k/22.5k [00:00<00:00, 21.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1s675Wt46d0-TqoHPqgM46-kVa4E8_m-Y
To: /content/2class-balanced-train.txt
100% 89.9k/89.9k [00:00<00:00, 32.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NKngN4pBT7nV9y6ALW6vI-w972r918-H
To: /content/NEGATIVE.txt
100% 253/253 [00:00<00:00, 226kB/s]
Downloading...
From: https://drive.google.com/uc?id=1MnyTd18BaP-KVNc2T_q6lmmek5KGVdVo
To: /content/reviews.tsv
39.9MB [00:00, 85.9MB/s]


In [ ]:
!pip install pyarabic

     |████████████████████████████████| 112kB 4.7MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.8-cp36-none-any.whl size=110609 sha256=61fd324f866085f0d536b88c6294342ec46520feb87180543ebd885b2aafb38e
  Stored in directory: /root/.cache/pip/wheels/ca/fa/26/d82414a6635b3ee82bee0729bfdb1dc8d09879742206e004bb
Successfully built pyarabic


In [ ]:
import pandas as pd
import numpy as np
import re
import pyarabic.araby as araby
import os
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras 

In [ ]:
reviews= pd.read_csv('reviews.tsv', sep='\t', names=['rating', 'review_id', 'user_id', 'book_id', 'review'])

In [ ]:
reviews.head()

,rating,review_id,user_id,book_id,review
0,4,338670838,7878381,13431841,"""عزازيل الذي صنعناه ،الكامن في أنفسنا"" يذكرني..."
1,4,39428407,1775679,3554772,من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...
2,4,32159373,1304410,3554772,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...
3,1,442326656,11333112,3554772,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...
4,5,46492258,580165,3554772,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...


In [ ]:
d=reviews[0:5].review.values

In [ ]:
d[0]

' "عزازيل الذي صنعناه ،الكامن في أنفسنا" يذكرني يوسف زيدان بــ بورخس في استخدامه لحيلته الفنية،وخداع القاريء بأن الرواية ترجمة لمخطوط قديم. الهوامش المخترعة و اختلاق وجود مترجـِم عاد بي إلى بورخس و هوامشه و كتَّابه الوهميين. هذه أولى قراءاتي ليوسف زيدان ،وهو عبقري في السرد ويخلقُ جوَّا ساحرا متفرداً يغرقك في المتعة. هُنا يتجلى الشكُّ الراقي الممزوج بانسانية هيبا الفاتنة ربما تم تناول فكرة الرواية قبلاً ،ولكن هنا تفرداً و عذوبة لا تُقارن بنصٍ آخر كنتُ أودُّ لو صيغت النهاية بطريقة مختلفة فقد جاءت باردة لا تتناسب مع رواية خُطَّت بهذا الشغف . ولذا لا أستطيع منح الرواية خمس نجوم ،وإن كانت تجربة قرائية متفردة وممتعة. '

In [ ]:
text=araby.strip_tashkeel(d[0])

In [ ]:
text=re.sub('[^\w]+|_]', ' ', text)

In [ ]:
text

' عزازيل الذي صنعناه الكامن في أنفسنا يذكرني يوسف زيدان بــ بورخس في استخدامه لحيلته الفنية وخداع القاريء بأن الرواية ترجمة لمخطوط قديم الهوامش المخترعة و اختلاق وجود مترجـم عاد بي إلى بورخس و هوامشه و كتابه الوهميين هذه أولى قراءاتي ليوسف زيدان وهو عبقري في السرد ويخلق جوا ساحرا متفردا يغرقك في المتعة هنا يتجلى الشك الراقي الممزوج بانسانية هيبا الفاتنة ربما تم تناول فكرة الرواية قبلا ولكن هنا تفردا و عذوبة لا تقارن بنص آخر كنت أود لو صيغت النهاية بطريقة مختلفة فقد جاءت باردة لا تتناسب مع رواية خطت بهذا الشغف ولذا لا أستطيع منح الرواية خمس نجوم وإن كانت تجربة قرائية متفردة وممتعة '

**extracting the train and test data from corpus**

In [ ]:
file1=open('2class-balanced-train.txt', 'r')
train_idx = [int(x) for x in file1.read().split('\n')]
file1.close()

file2=open('2class-balanced-test.txt', 'r')
test_idx=[int(x) for x in file2.read().split('\n')]
file2.close()

In [ ]:
train_reviews = reviews.iloc[train_idx, [0, -1]]
test_reviews = reviews.iloc[test_idx, [0, -1]]
train_reviews.rating = train_reviews.rating.apply(lambda x : 0 if x<3 else 1)
test_reviews.rating = test_reviews.rating.apply(lambda x : 0 if x<3 else 1)
reviews = pd.concat([train_reviews, test_reviews])

In [ ]:
x_train = train_reviews.review.values
x_test  = test_reviews.review.values
y_train = train_reviews.rating.values
y_test  = test_reviews.rating.values

**ازاله التشكيل وعلامات الترقيم**

In [ ]:
x_train = [re.sub('[^\w]+|_]', ' ', araby.strip_tashkeel(review)) for review in x_train]

In [ ]:
x_test = [re.sub('[^\w]+|_]', ' ', araby.strip_tashkeel(review)) for review in x_test]

** Remove stop words but not the Negation words**



In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = stopwords.words('arabic')

file1=open('NEGATIVE.txt', 'r')
neg = [x for x in file1.read().split('\n')]
file1.close()

n = len(stop_words)

for word in neg:
  if word in stop_words:
    stop_words.remove(word)

print('{} stop words has been deleted'.format(n-len(stop_words)))

16 stop words has been deleted


**vectorizing the corpus and constructing a bag of words**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=2000, stop_words=stop_words, ngram_range=(1, 2))
x_train1 = cv.fit_transform(x_train).toarray()
x_test1 = cv.transform(x_test).toarray()

**Logistic regression model**

In [ ]:
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(max_iter=100)
model1.fit(x_train1, y_train)
y_pred1 = model1.predict(x_test1)
accuracy = accuracy_score(y_test, y_pred1)
print('Logistic regression model accuracy : {}'.format(accuracy))
print('\n\n')

Logistic regression model accuracy : 0.7715936739659367





/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


**Naive bayes model**

In [ ]:
from sklearn.naive_bayes import GaussianNB

model2 = GaussianNB()
model2.fit(x_train1, y_train)
y_pred2 = model2.predict(x_test1)
print('Naive Bayes Model accuracy : {}'.format(accuracy_score(y_test, y_pred2)))

Naive Bayes Model accuracy : 0.7305352798053528


**TF-IDF Vectorization**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 2), max_features=2000, )
x_train2 = tfidf.fit_transform(x_train).toarray()
x_test2  = tfidf.transform(x_test).toarray()

In [ ]:
model3 = LogisticRegression(max_iter=200)
model3.fit(x_train2, y_train)
y_pred3 = model3.predict(x_test2)
print('Logistic regression Model with TF-IDF representation accuracy : {}'.format(accuracy_score(y_test, y_pred3)))

Logistic regression Model with TF-IDF representation accuracy : 0.7871046228710462
